In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

# Função 1

In [2]:
from sympy import symbols

variaveis = list(symbols('x1:8'))

print("Variáveis da função 1:")
display(variaveis)

Variáveis da função 1:


[x1, x2, x3, x4, x5, x6, x7]

In [3]:
f_x = sum([100 * (variaveis[i+1] - variaveis[i]**2)**2 + (1 - variaveis[i])**2 for i in range(len(variaveis)-1)])

print("Função 1:")
display(f_x)

Função 1:


(1 - x1)**2 + (1 - x2)**2 + (1 - x3)**2 + (1 - x4)**2 + (1 - x5)**2 + (1 - x6)**2 + 100*(-x1**2 + x2)**2 + 100*(-x2**2 + x3)**2 + 100*(-x3**2 + x4)**2 + 100*(-x4**2 + x5)**2 + 100*(-x5**2 + x6)**2 + 100*(-x6**2 + x7)**2

In [4]:
from algoritmos import calcula_gradiente

grad_f_x = calcula_gradiente(f_x, variaveis)

print("Vetor gradiente da função 1:")
display(grad_f_x)

Vetor gradiente da função 1:


[-400*x1*(-x1**2 + x2) + 2*x1 - 2,
 -200*x1**2 - 400*x2*(-x2**2 + x3) + 202*x2 - 2,
 -200*x2**2 - 400*x3*(-x3**2 + x4) + 202*x3 - 2,
 -200*x3**2 - 400*x4*(-x4**2 + x5) + 202*x4 - 2,
 -200*x4**2 - 400*x5*(-x5**2 + x6) + 202*x5 - 2,
 -200*x5**2 - 400*x6*(-x6**2 + x7) + 202*x6 - 2,
 -200*x6**2 + 200*x7]

In [5]:
from algoritmos import calcula_hessiana

hess_f_x = calcula_hessiana(f_x, variaveis)

print("Matriz Hessiana da função 1:")
display(hess_f_x)

Matriz Hessiana da função 1:


Matrix([
[1200*x1**2 - 400*x2 + 2,                   -400*x1,                         0,                         0,                         0,                         0,       0],
[                -400*x1, 1200*x2**2 - 400*x3 + 202,                   -400*x2,                         0,                         0,                         0,       0],
[                      0,                   -400*x2, 1200*x3**2 - 400*x4 + 202,                   -400*x3,                         0,                         0,       0],
[                      0,                         0,                   -400*x3, 1200*x4**2 - 400*x5 + 202,                   -400*x4,                         0,       0],
[                      0,                         0,                         0,                   -400*x4, 1200*x5**2 - 400*x6 + 202,                   -400*x5,       0],
[                      0,                         0,                         0,                         0,                   -400*x5, 12

## Estudo da função
A função 1 é uma generalização multidimensional da função de Rosenbrock, uma função usada para testar algoritmos de otimização. A função 1 é definida por:

\begin{equation}
f(\mathbf{x}) = \sum_{i=1}^{6} 100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2
\end{equation}

s. a.

\begin{equation}
x \in \mathbb{R}^7
\end{equation}

Ela é uma função não-convexa e, como na função de Rosenbrock, o mínimo global é localizado em um vale estreito e longo.

In [8]:
import numpy as np
import plotly.graph_objects as go
from sympy import lambdify

f_x_func = lambdify(variaveis, f_x, 'numpy')

x1_vals = np.linspace(-2, 2, 100)
x2_vals = np.linspace(-2, 2, 100)
x1, x2 = np.meshgrid(x1_vals, x2_vals)


fixed_values = [0] * 5
f_vals = np.array([
    f_x_func(x1_val, x2_val, *fixed_values)
    for x1_val, x2_val in zip(np.ravel(x1), np.ravel(x2))
]).reshape(x1.shape)

fig = go.Figure(data=[go.Surface(z=f_vals, x=x1, y=x2, colorscale=[
        [0.0, 'blue'],
        [0.002, 'cyan'],
        [0.05, 'yellow'],
        [0.1, 'orange'],
        [1.0, 'red']
    ], colorbar=dict(title="Valores"))])
fig.update_layout(
    title="Projeção da função 1 no plano x1-x2 (x3,..., x7 = 0)",
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='f(x1, x2, 0, ..., 0)'
    ),
    width=800,
    height=800
)

fig.show()

In [7]:
f_x_2d = f_x.subs({variaveis[i]: 0 for i in range(2, len(variaveis))})

f_numeric = lambdify([variaveis[0], variaveis[1]], f_x_2d, 'numpy')

x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
x_grid, y_grid = np.meshgrid(x, y)

z = f_numeric(x_grid, y_grid)

fig = go.Figure(data=go.Contour(
    z=z,
    x=x,
    y=y,
    colorscale=[
        [0.0, 'blue'],
        [0.05, 'cyan'],
        [0.2, 'yellow'],
        [0.3, 'orange'],
        [1.0, 'red']
    ],
    contours=dict(start=np.min(z), end=np.max(z), size=100)
))

fig.update_layout(
    title="Curvas de Nível da Função 1",
    xaxis_title="x1",
    yaxis_title="x2",
    width=800,
    height=800
)

fig.show()

In [ ]:
from algoritmos import metodo_do_gradiente, metodo_de_newton, metodo_de_quase_newton
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['Método', 'Gamma', 'Eta', 'Iterações', 'Iterações Armijo', 'Ponto otimo'])

ponto =  np.array([0.02, -0.03, 0.04, -0.05, 0.06, -0.07, 0.08])

results = []

for gamma in [0.1, 0.25, 0.5, 0.9]:
    for eta in [0.1, 0.25, 0.5, 0.9]:
        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_do_gradiente(f_x, grad_f_x, variaveis, ponto, gamma, eta)

        results.append({
            'Método': 'Gradiente',
            'Gamma': gamma,
            'Eta': eta,
            'Iterações': n_iteracoes,
            'Iterações Armijo': n_iteracoes_armijo,
            'Ponto otimo': ponto_otimo,
        })

        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_newton(f_x, grad_f_x, hess_f_x, variaveis, ponto, gamma, eta)

        results.append({
            'Método': 'Newton',
            'Gamma': gamma,
            'Eta': eta,
            'Iterações': n_iteracoes,
            'Iterações Armijo': n_iteracoes_armijo,
            'Ponto otimo': ponto_otimo,
        })

        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_quase_newton(f_x, len(variaveis), grad_f_x, variaveis, ponto, gamma, eta, metodo='bfgs')

        results.append({
            'Método': 'Quase Newton (BFGS)',
            'Gamma': gamma,
            'Eta': eta,
            'Iterações': n_iteracoes,
            'Iterações Armijo': n_iteracoes_armijo,
            'Ponto otimo': ponto_otimo,
        })

        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_quase_newton(f_x, len(variaveis), grad_f_x, variaveis, ponto, gamma, eta, metodo='dfp')

        results.append({
            'Método': 'Quase Newton (DFP)',
            'Gamma': gamma,
            'Eta': eta,
            'Iterações': n_iteracoes,
            'Iterações Armijo': n_iteracoes_armijo,
            'Ponto otimo': ponto_otimo,
        })

df = pd.concat([df, pd.DataFrame(results)], ignore_index=True)

In [12]:
df.to_csv('datasets/resultados_funcao_1.csv', index=False)

In [ ]:
df = pd.read_csv('datasets/resultados_funcao_1.csv')

df_gradiente = df[df['Método'] == 'Gradiente'].sort_values(by='Iterações', ascending=True).head(1)
df_newton = df[df['Método'] == 'Newton'].sort_values(by='Iterações', ascending=True).head(1)
df_quase_newton_bfgs = df[df['Método'] == 'Quase Newton (BFGS)'].sort_values(by='Iterações', ascending=True).head(1)
df_quase_newton_dfp = df[df['Método'] == 'Quase Newton (DFP)'].sort_values(by='Iterações', ascending=True).head(1)

df_top3 = pd.concat([df_gradiente, df_newton, df_quase_newton_bfgs, df_quase_newton_dfp])
df_top3.to_csv('datasets/resultados_funcao_1_top3.csv', index=False)

## Melhores hiperparâmetros

In [14]:
df_top3

,Método,Gamma,Eta,Iterações,Iterações Armijo,Ponto otimo
0,Gradiente,0.1,0.10,1000,3790,[0.99549075 0.9910699 0.98204482 0.96447963 0...
37,Newton,0.5,0.25,20,23,[1. 1. 1. 0.99999999 0...
38,Quase Newton (BFGS),0.5,0.25,45,104,[1. 1. 1. 1. 0...
3,Quase Newton (DFP),0.1,0.10,1000,1600,[0.8798658 0.76971702 0.57763346 0.34680904 0...


In [22]:
import numpy as np
import pandas as pd

colunas = ['Algoritmo', 'Ponto inicial', '# de iteracoes', '# de cham. de armijo', 'Ponto otimo', 'Valor otimo', 'Erro de aproximação', 'Tempo de exec (s)']
df_base = pd.DataFrame(columns=colunas)

pontos = [
  np.array([0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]),
  np.array([2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]),
  np.array([2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]),
  np.array([-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]),
  np.array([7.9, 6.9, 5.9, 4.9, 3.9, 2.9, 2.1]),
  np.array([-7.9, -6.9, -5.9, -4.9, -3.9, -2.9, -2.1]),
  ]

print("Pontos a serem testados:")
display(pontos)

Pontos a serem testados:


[array([0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]),
 array([2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]),
 array([2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]),
 array([-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]),
 array([7.9, 6.9, 5.9, 4.9, 3.9, 2.9, 2.1]),
 array([-7.9, -6.9, -5.9, -4.9, -3.9, -2.9, -2.1])]

### Gradiente

In [26]:
from algoritmos import metodo_do_gradiente
from utils import substitui_variaveis_funcao, substitui_variaveis_gradiente
import time

df_gradiente = df_base.copy()

for ponto in pontos:
    tic = time.time()
    ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_do_gradiente(f_x, grad_f_x, variaveis, ponto, gamma=0.5, eta=0.25, maximo_iteracoes=2500)
    tac = time.time()
    valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
    erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
    df_gradiente = pd.concat([df_gradiente, pd.DataFrame([['gradiente', ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao, tac - tic]], columns=colunas)])

df_gradiente

Método do Gradiente: 100%|██████████| 2500/2500 [04:04<00:00, 10.23it/s]


,Algoritmo,Ponto inicial,# de iteracoes,# de cham. de armijo,Ponto otimo,Valor otimo,Erro de aproximação,Tempo de exec (s)
0,gradiente,"[0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]",2500,26718,"[0.9998318265444419, 0.9996662689294042, 0.999...",3.90411988529089e-5,0.010153,254.496436
0,gradiente,"[2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]",2500,30966,"[1.0291344067421868, 1.0592551732579927, 1.122...",2.77659297379238,0.946361,385.305093
0,gradiente,"[2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]",2500,30977,"[1.0291693732891456, 1.059327359883787, 1.1225...",2.78653699257475,1.772477,385.934081
0,gradiente,"[-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]",2500,26228,"[0.9979632285274371, 0.9959010149218626, 0.991...",0.00552276113027757,0.091305,255.680764
0,gradiente,"[7.9, 6.9, 5.9, 4.9, 3.9, 2.9, 2.1]",2500,26886,"[1.0024794595633915, 1.0050171166762718, 1.010...",0.00927194702553367,0.178349,338.335089
0,gradiente,"[-7.9, -6.9, -5.9, -4.9, -3.9, -2.9, -2.1]",2500,26230,"[0.997959107922576, 0.9959445798572064, 0.9918...",0.00548040289381809,0.099610,244.393042


### Newton

In [27]:
from algoritmos import metodo_de_newton
df_newton = df_base.copy()

for ponto in pontos:
    tic = time.time()
    ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_newton(f_x, grad_f_x, hess_f_x, variaveis, ponto, gamma=0.5, eta=0.25, maximo_iteracoes=2500)
    tac = time.time()
    valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
    erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
    df_newton = pd.concat([df_newton, pd.DataFrame([['newton', ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao, tac - tic]], columns=colunas)])

display(df_newton)


Método de Newton:   0%|          | 0/2500 [00:00<?, ?it/s]

Método de Newton:   1%|▏         | 33/2500 [00:00<01:01, 40.42it/s]


,Algoritmo,Ponto inicial,# de iteracoes,# de cham. de armijo,Ponto otimo,Valor otimo,Erro de aproximação,Tempo de exec (s)
0,newton,"[0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]",10,10,"[1.00000000037853, 1.000000000758719, 1.000000...",2.00733273811594e-16,9.428158e-08,0.309976
0,newton,"[2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]",52,72,"[1.000000000342515, 1.0000000006848606, 1.0000...",5.32486648614157e-16,8.154636e-07,1.475988
0,newton,"[2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]",52,71,"[1.0000000001893383, 1.0000000003791405, 1.000...",7.05021392625166e-17,2.078573e-07,1.463945
0,newton,"[-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]",31,41,"[0.9999999999999986, 0.9999999999999971, 0.999...",3.07832014144705e-27,5.540411e-13,0.730448
0,newton,"[7.9, 6.9, 5.9, 4.9, 3.9, 2.9, 2.1]",40,51,"[1.000000000000617, 1.0000000000012215, 1.0000...",2.87903567438760e-20,6.795942e-09,1.097820
0,newton,"[-7.9, -6.9, -5.9, -4.9, -3.9, -2.9, -2.1]",33,47,"[0.9999999997286072, 0.9999999994549466, 0.999...",2.24012542177507e-16,4.008009e-07,0.817564


### Quase-Newton

In [28]:
from algoritmos import metodo_de_quase_newton

df_quase_newton = df_base.copy()

for metodo in [
    ('bfgs', { 'gamma': 0.5, 'eta': 0.25 }),
    ('dfp', { 'gamma': 0.5, 'eta': 0.25 })
]:
    for ponto in pontos:
        tic = time.time()
        ponto_otimo, n_iteracoes, n_iteracoes_armijo = metodo_de_quase_newton(f_x, len(variaveis), grad_f_x, variaveis, ponto, gamma=metodo[1]['gamma'], eta=metodo[1]['eta'], metodo=metodo[0], maximo_iteracoes=2500)
        tac = time.time()
        valor_otimo = substitui_variaveis_funcao(f_x, variaveis, ponto_otimo)
        erro_aproximacao = np.linalg.norm(substitui_variaveis_gradiente(grad_f_x, variaveis, ponto_otimo))
        df_quase_newton = pd.concat([df_quase_newton, pd.DataFrame([[f'quase-newtown ({metodo[0]})', ponto, n_iteracoes, n_iteracoes_armijo, ponto_otimo, valor_otimo, erro_aproximacao, tac - tic]], columns=colunas)])

display(df_quase_newton)

Método de Quase Newton (dfp): 100%|██████████| 2500/2500 [01:21<00:00, 30.53it/s]


,Algoritmo,Ponto inicial,# de iteracoes,# de cham. de armijo,Ponto otimo,Valor otimo,Erro de aproximação,Tempo de exec (s)
0,quase-newtown (bfgs),"[0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]",21,84,"[1.000000000005972, 0.9999999999837773, 0.9999...",5.07681903556742e-18,8.126788e-08,1.190328
0,quase-newtown (bfgs),"[2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]",71,157,"[-0.9917225726392842, 0.9935553930435241, 0.99...",3.98360053642485,6.024386e-07,2.136011
0,quase-newtown (bfgs),"[2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]",37,107,"[1.0000000000427698, 0.9999999999997548, 0.999...",3.49476842206936e-18,7.764041e-08,1.479190
0,quase-newtown (bfgs),"[-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]",83,179,"[1.0000000000630902, 1.000000000068437, 1.0000...",8.70890734155018e-18,1.356573e-07,2.462945
0,quase-newtown (bfgs),"[7.9, 6.9, 5.9, 4.9, 3.9, 2.9, 2.1]",70,159,"[0.999999999865711, 0.9999999996050241, 0.9999...",1.86912225349928e-16,5.298994e-07,2.419183
0,quase-newtown (bfgs),"[-7.9, -6.9, -5.9, -4.9, -3.9, -2.9, -2.1]",89,183,"[1.0000000000154914, 0.9999999997744048, 0.999...",3.04006284115260e-17,2.562035e-07,2.542467
0,quase-newtown (dfp),"[0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]",2500,6659,"[0.9998729446463165, 1.0008398007511765, 1.001...",0.000377528111958072,7.784850e-01,95.941846
0,quase-newtown (dfp),"[2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]",2500,6867,"[-0.3000595918241857, 0.0789218236166513, -0.0...",10.9319323993293,4.783684e+01,81.175183
0,quase-newtown (dfp),"[2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]",2500,6467,"[0.628387565873067, 0.39177429761133553, 0.163...",4.22203309008577,7.322212e+00,77.228447
0,quase-newtown (dfp),"[-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]",2500,6976,"[0.8746024990486859, 0.6628190626375691, 0.293...",11.3872204207371,6.091627e+01,83.477119


## Tabela geral

In [29]:
df_all = pd.concat([df_gradiente, df_newton, df_quase_newton])
df_all.to_csv('datasets/resultados_funcao_1_todos.csv', index=False)
df_all

,Algoritmo,Ponto inicial,# de iteracoes,# de cham. de armijo,Ponto otimo,Valor otimo,Erro de aproximação,Tempo de exec (s)
0,gradiente,"[0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]",2500,26718,"[0.9998318265444419, 0.9996662689294042, 0.999...",3.90411988529089e-5,1.015297e-02,254.496436
0,gradiente,"[2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]",2500,30966,"[1.0291344067421868, 1.0592551732579927, 1.122...",2.77659297379238,9.463606e-01,385.305093
0,gradiente,"[2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]",2500,30977,"[1.0291693732891456, 1.059327359883787, 1.1225...",2.78653699257475,1.772477e+00,385.934081
0,gradiente,"[-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]",2500,26228,"[0.9979632285274371, 0.9959010149218626, 0.991...",0.00552276113027757,9.130538e-02,255.680764
0,gradiente,"[7.9, 6.9, 5.9, 4.9, 3.9, 2.9, 2.1]",2500,26886,"[1.0024794595633915, 1.0050171166762718, 1.010...",0.00927194702553367,1.783486e-01,338.335089
0,gradiente,"[-7.9, -6.9, -5.9, -4.9, -3.9, -2.9, -2.1]",2500,26230,"[0.997959107922576, 0.9959445798572064, 0.9918...",0.00548040289381809,9.960971e-02,244.393042
0,newton,"[0.9, 1.1, 0.9, 1.1, 0.9, 1.1, 0.9]",10,10,"[1.00000000037853, 1.000000000758719, 1.000000...",2.00733273811594e-16,9.428158e-08,0.309976
0,newton,"[2.9, 2.1, 3.9, 4.9, 5.9, 6.9, 7.9]",52,72,"[1.000000000342515, 1.0000000006848606, 1.0000...",5.32486648614157e-16,8.154636e-07,1.475988
0,newton,"[2.1, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9]",52,71,"[1.0000000001893383, 1.0000000003791405, 1.000...",7.05021392625166e-17,2.078573e-07,1.463945
0,newton,"[-2.9, -2.1, -3.9, -4.9, -5.9, -6.9, -7.9]",31,41,"[0.9999999999999986, 0.9999999999999971, 0.999...",3.07832014144705e-27,5.540411e-13,0.730448
